# LSTM + Dense 모델을 작성해 주가 예측
삼성전자 증권표준코드 005930 를 이용해서 주가 예측




In [ ]:
!pip install finance-datareader

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import warnings
warnings.filterwarnings('ignore')

In [ ]:
STOCK_CODE='005930'
stock_data=fdr.DataReader(STOCK_CODE)

print(stock_data.head(3))
print(stock_data.tail(3))
print()

print('상관관계 : ',stock_data.corr())
print()

stock_data.reset_index(inplace=True)
print(stock_data.head(3))
print()

stock_data.drop(['Change'], axis='columns',inplace=True)
print(stock_data.head(3))
print()

print(stock_data.info())

# OLS, 분류- 집단위 나눠질때는 집단 평균차이를 확인할때는 ttest, anova, 빈도차이는 독립성? 선호도- 카이스퀘어,
# 가설검정, 기술통계는 반드시 집어넣주어야 한다- 데이터의 전체적인 내용을 확인해야 되니까
# 추론통계로 들어와서 regression, classification 이 있는데 가설검정이 있다치면 가설검정도 넣어주고, 딥러닝을 사용하기에 리그레션도 LSTM없이 할수 있음
# 텐서플로또한 LSTM을 넣어서 운영할 수 있다.

Date열을 이용해 연도별 주가 변동 시각화

Date열을 연,월,일로 분리해 새로운 열을 추가

In [ ]:
stock_data['year']=stock_data['Date'].dt.year
stock_data['month']=stock_data['Date'].dt.month
stock_data['day']=stock_data['Date'].dt.day
print(stock_data.head(3))
print(stock_data.shape)

df=stock_data.loc[stock_data['year']>=2000]
plt.figure(figsize=(6,4))
sns.lineplot(y=df['Close'],x=df.year)
plt.xlabel('time')
plt.ylabel('price')
plt.show()

In [ ]:
# 전처리 계속 : Open High Low Close Volumne
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scale_cols = ['Open','High','Low','Close','Volume']
df_scaled = scaler.fit_transform(stock_data[scale_cols])
df_scaled= pd.DataFrame(df_scaled)
df_scaled.columns=scale_cols
print(df_scaled.head(3))

col_close=['Close']
close_scaled=scaler.fit_transform(stock_data[col_close])
print('스케일 값: ', close_scaled[:5].ravel())
print('스케일 원복 값: ', scaler.inverse_transform(close_scaled[:5]).ravel())

# 과거 20일을 기준으로 그 다음날의 종가를 예측, 전체 데이터는 과거 200일 기준, 예측 기준은 과거 20일 사용
TEST_SIZE=200
train=df_scaled[:-TEST_SIZE]
test=df_scaled[-TEST_SIZE:]
print(train.shape) # (5800, 4)
print(test.shape) # (200, 4)

# dataset 작성 함수
def make_dataset_func(data, label, window_size=20):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i + window_size]))
        label_list.append(np.array(label.iloc[i + window_size]))
    return np.array(feature_list), np.array(label_list) #순차적으로 20일동안의 data를 묶어 label과 함께 대응하는 label과 함께 반환

feature_cols = ['Open','High','Low','Volume']
label_cols=['Close']

train_feature = train[feature_cols]
train_label = train[label_cols]

test_feature = test[feature_cols]
test_label = test[label_cols]
print(train_feature[:3])
print(train_label[:3])
print(train_feature.shape, train_label.shape, test_feature.shape, test_label.shape)

train_feature,train_label = make_dataset_func(train_feature, train_label)
print(train_feature[:2],train_label[:2])
print()

# train_test split
from sklearn.model_selection import train_test_split
x_train,x_test, y_train,y_test = train_test_split(train_feature,train_label,test_size=0.2,random_state=12,shuffle=False)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape) #(4624, 20, 4) (1156, 20, 4) (4624, 1) (1156, 1)

# test
test_feature,test_label = make_dataset_func(test_feature, test_label)
print(test_feature[:2],test_label[:2])
print(test_feature.shape,test_label.shape)


In [ ]:
# model
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint

model=Sequential()
model.add(LSTM(units=16,activation='tanh',input_shape=(train_feature.shape[1],train_feature.shape[2]),return_sequences=False)) # many-to-one #하나로 나가야 하기때문에 return sequences는 false, 많이 나가면 many to many

model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=1))
print( model.summary())


In [ ]:
from keras.losses import Huber
model.compile(optimizer='adam',loss=Huber(),metrics=['mse'])
# Huber loss는 모든 지점에서 미분이 가능하면서 이상치에 강건한(robust) 성격을 보이는 loss function이다 - 이상치에 덜 민감

es=EarlyStopping(monitor='val_loss', patience=30)
chkPoint=ModelCheckpoint('tf33.hdf5', monitor='val_loss',save_best_only=True,mode='auto',verbose=0)
history=model.fit(x_train,y_train,epochs=100, batch_size=8, validation_data=(x_test,y_test),verbose=2,callbacks=[es,chkPoint])


In [ ]:
plt.figure(figsize=(6,4))
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='val_loss',c='red')
plt.legend()
plt.show()

In [ ]:
import keras
model= keras.models.load_model('tf33.hdf5')

pred=model.predict(test_feature,verbose=0)

from sklearn.metrics import r2_score
print('r2_score : ',r2_score(test_label,pred))

print('pred : ',np.round(pred[:10].flatten(),2))
print('real : ',np.round(test_label[:10].flatten(),2))
print('pred :', scaler.inverse_transform(pred[:10]).flatten())
print('real :', scaler.inverse_transform(test_label[:10]).flatten())

#실제값, 예측값 시각화
plt.figure(figsize=(6,4))
plt.plot(test_label[:20],label='real')
plt.plot(pred[:20].flatten(),label='pred',c='red')
plt.legend()
plt.show()
#실제값과 예측값이 패턴을 잘 찾고 있다.
